# Building a Multi-Agent System with LangGraph

- We'll build a system crew of agents to be lead by a supervisor agent.
- The Crew will consists of 
    - an agent capable of solving basic maths problems, 
    - an agent that will handle text edit requests,
    - an agent that will communicate with the user.
- The supervisor agent will determine which agent will be called next given the user query and agent work history.

Source: LangGraph `https://docs.langchain.com/oss/python/langchain/supervisor`
source: `https://medium.com/@ipeksahbazoglu/building-a-multi-agent-system-with-langgraph-and-gemini-1e7d7eab5c12`
- The supervisor pattern is a multi-agent architechture where a central supervisor agent coordinates specialized worker agents.
 

In [55]:
import os
from langchain_aws import ChatBedrock
from dotenv import load_dotenv
load_dotenv()

True

**Initialize the LLM**

In [56]:
AWS_REGION = os.getenv("AWS_REGION_NAME")
def get_bedrock_llm():
    """Initializes and returns the Bedrock LLM client."""
    MODEL_ID = "arn:aws:bedrock:us-east-1:487288114819:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0"
    llm = ChatBedrock(
        region=AWS_REGION, 
        model=MODEL_ID,
        provider="anthropic",
        model_kwargs={
            "max_tokens": 1000,
            "temperature": 0.1,
        }
    )
    return llm

In [57]:
llm = get_bedrock_llm()

In [58]:
# test-1 (llm invocation)
response = llm.invoke("What is the capital of Kerela?")
print(response.content)

The capital of Kerala is **Thiruvananthapuram** (also known as Trivandrum).


**Defining the set of tools**

In [59]:
# tools for agent solving basic math problems
from langchain.tools import tool

@tool
def add(a: float, b: float) -> float:
    """Adds two numbers."""
    return a + b 

@tool
def subtract(a: float, b: float) -> float:
    """Subtracts two numbers."""
    return a - b   

@tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers."""
    return a * b

@tool
def divide(a: float, b: float):
    """Divides two numbers."""
    if b == 0:
        return "Error: Division by zero is undefined."
    return a / b

@tool
def square(a: float) -> float:
    """Returns the square of a number."""
    a = int(a)
    return a * a

math_toolkit = [add, subtract, multiply, divide, square]

In [60]:
# tools for agent handling text edit requests
@tool
def make_uppercase(text: str) -> str:
    """Converts text to uppercase."""
    return text.upper()

@tool
def make_lowercase(text: str) -> str:
    """Converts text to lowercase."""
    return text.lower()

@tool
def reverse_text(text: str) -> str:
    """Reverses the given text."""
    return text[::-1]  

@tool
def add_smiley(text: str) -> str:
    """Adds a smiley face to the end of the text."""
    return text + " 😊"

text_toolkit = [make_uppercase, make_lowercase, reverse_text, add_smiley]

**Create the Agents**
- Two Agents: *math solver* and *text handler*
- We create a helper functions to create a tool calling agents given the LLM, toolkit and system prompt. These tool agents will receive an input task and context if necessary from a supervisor agent.

In [61]:
from langchain.agents import create_agent

# agent - 1 math_solver
SYSTEM_PROMPT_MATH = """You are a mathematical assistant.
Use the provided tools to answer the user's math-related queries accurately.
If you do not have a tool to answer the query, respond with 'I cannot help with that.'"""
math_agent = create_agent(
    llm,
    tools=math_toolkit,
    system_prompt=SYSTEM_PROMPT_MATH, 
)

# agent - 2 text_handler
SYSTEM_PROMPT_TEXT = """You are a text editing assistant.
Use the provided tools to perform text manipulations as requested by the user.
If you do not have a tool to answer the query, respond with 'I cannot help with that.'"""
text_agent = create_agent(
    llm,
    tools=text_toolkit,
    system_prompt=SYSTEM_PROMPT_TEXT,
)

In [62]:
# test -2 (math agent)
query = "What is the square of 15?"
query1 = "What is the log of 100?"

for step in math_agent.stream(
    {"messages": [{"role": "user", "content": query1}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================

I cannot help with that.

The tools I have available are for basic arithmetic operations (addition, subtraction, multiplication, division) and squaring numbers. I don't have a logarithm function available to calculate the log of 100.

However, if you're asking about the common logarithm (base 10), log₁₀(100) = 2, since 10² = 100. If you're asking about the natural logarithm (base e), ln(100) ≈ 4.605.


In [63]:
# test -3 (text agent)
query = "Change the following text to uppercase: 'hello world'."

for step in text_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  make_uppercase (toolu_bdrk_01JMWdmnMzpLV975Ato4y4ni)
 Call ID: toolu_bdrk_01JMWdmnMzpLV975Ato4y4ni
  Args:
    text: hello world
================================= Tool Message =================================
Name: make_uppercase

HELLO WORLD
================================== Ai Message ==================================

The text has been converted to uppercase: **HELLO WORLD**


We will a communication agent.
this agent will read the conversation history and communicate results to the user.

In [64]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder

class CommunicationOutput(BaseModel):
    message: AIMessage = Field(description="clear and friendly message to communicate to the user.")
    
communication_output_parser = JsonOutputParser(pydantic_object=CommunicationOutput)

SYSTEM_PROMPT_TEMPLATE = PromptTemplate(
    template="""You are talkative and friendly assistant that summarizes the work done by other agents as history and communicates it to the user.
    Summarize all the outputs from other agents in the history and provide a clear and friendly message to the user.
    The agent work history is as follows: {agent_history}
    """,
    input_variables=["agent_history"])
SYSTEM_MESSAGE_PROMPT = SystemMessagePromptTemplate(prompt=SYSTEM_PROMPT_TEMPLATE)
prompt = ChatPromptTemplate.from_messages([SYSTEM_MESSAGE_PROMPT,
                                           MessagesPlaceholder(variable_name="messages")])
comms_agent = (prompt|llm)
comms_agent


ChatPromptTemplate(input_variables=['agent_history', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[l

In [65]:
from langchain_core.messages import HumanMessage
# test -4 (communication agent)
text_output = text_agent.invoke({
    "messages": [HumanMessage(content="Hi! can you capitalize this: banna")], 
    "agent_history":[]
})
comms_output = comms_agent.invoke({
    "messages": [HumanMessage(content="Hi!how are you? can you capitalize this: banna")],
    "agent_history":[text_output]
})


In [66]:
text_output

{'messages': [HumanMessage(content='Hi! can you capitalize this: banna', additional_kwargs={}, response_metadata={}, id='e7ac8447-452c-4ef4-8252-a62535714952'),
  AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 782, 'completion_tokens': 54, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 836}, 'stop_reason': 'tool_use', 'model_id': 'arn:aws:bedrock:us-east-1:487288114819:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0'}, response_metadata={'usage': {'prompt_tokens': 782, 'completion_tokens': 54, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 836}, 'stop_reason': 'tool_use', 'model_id': 'arn:aws:bedrock:us-east-1:487288114819:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0', 'model_provider': 'bedrock', 'model_name': 'arn:aws:bedrock:us-east-1:487288114819:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0'}, id='lc_run--aa945da7-e06b-4510-be2d-1f0142399a2

In [67]:
text_output['messages'][-1].content

'The capitalized text is: **BANNA**'

In [68]:
comms_output

AIMessage(content='Hello! I\'m doing great, thank you for asking! 😊\n\nBased on the work history, I can see that you previously asked to capitalize "banna", and our system successfully processed that request.\n\n**The capitalized version is: BANNA**\n\nThe agent used a text transformation tool to convert all the letters in "banna" to uppercase, resulting in "BANNA".\n\nIs there anything else you\'d like me to help you with today?', additional_kwargs={'usage': {'prompt_tokens': 1124, 'completion_tokens': 101, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1225}, 'stop_reason': 'end_turn', 'model_id': 'arn:aws:bedrock:us-east-1:487288114819:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0'}, response_metadata={'usage': {'prompt_tokens': 1124, 'completion_tokens': 101, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1225}, 'stop_reason': 'end_turn', 'model_id': 'arn:aws:bedrock:us-east-1:487288114819:inference-pr

**Create the Surpervisor Agent**

In [69]:
from enum import Enum
members = ['Calculator', 'TextHandler', 'Communicator']

member_options = {member: member for member in members}
print(member_options)

MemberEnum = Enum('MemberEnum', member_options)
print(MemberEnum)

class SupervisorOutput(BaseModel):
    next: MemberEnum = MemberEnum.Communicator

{'Calculator': 'Calculator', 'TextHandler': 'TextHandler', 'Communicator': 'Communicator'}
<enum 'MemberEnum'>


In [70]:
SYSTEM_PROMPT = ("""You are a supervisor agent tasked with managing a conversation between 
                 the crew of agents : {members}. Given the following user request, and crew responses
                 respond with the next agent to handle the user request based on the context.
                 Each worker agent will perform a task and respond with their results and status. When finished with 
                 the task, route to communicate to deliver the result to user. Given the conversation history below, who should act next?
                 Select from the following options: {options}
                 \n{format_instructions}
                 """)

supervisor_output_parser = JsonOutputParser(pydantic_object=SupervisorOutput)
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="messages"),
    MessagesPlaceholder(variable_name="agent_history")
]).partial(options=str(members), members=", ".join(members), format_instructions=supervisor_output_parser.get_format_instructions())

supervisor_chain = (prompt | llm | supervisor_output_parser)


In [71]:
# Create Crew Node innvocation function

def crew_nodes(state, crew_menber, name):
    input = {'messages': state['messages'][-1], 'agent_history': state['agent_history']}
    result = crew_menber.invoke(input)
    # print(result)
    return {"agent_histroy": [AIMessage(content=result['messages'][-1].content,additional_kwargs={'intermediate_steps': result.get('intermediate_steps', [])}, name=name)]}

def comms_node(state):
    input = {'messages': state['messages'][-1], 'agent_history': state['agent_history']}
    result = comms_agent.invoke(input) 
    return {"messages":[result]}

**Build Graph**

In [72]:
import operator
from typing import Annotated, Sequence, TypedDict
import functools
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, END

# define the graph state
# the agent state is the input to each node in the graph
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add] # the annotation tells the graph that the new messages will always be added to the current states
    next: str
    agent_history: Annotated[Sequence[BaseMessage], operator.add]
    
# prepare the agent node
math_node = functools.partial(crew_nodes, crew_menber=math_agent, name='Calculator')
text_node = functools.partial(crew_nodes, crew_menber=text_agent, name='TextHandler')

workflow = StateGraph(AgentState)
workflow.add_node('Calculator', math_node)
workflow.add_node('TextHandler', text_node) 
workflow.add_node('Communicator', comms_node)
workflow.add_node('Supervisor', supervisor_chain)



In [73]:
workflow.add_edge('Calculator', 'Supervisor')
workflow.add_edge('TextHandler', 'Supervisor')  
workflow.add_edge('Communicator', END)

workflow.add_conditional_edges("Supervisor", lambda x: x["next"], member_options)
# Finally, add entrypoint
workflow.set_entry_point("Supervisor")

graph = workflow.compile()

In [74]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Finally, compile the graph! 
# This compiles it into a LangChain Runnable,
graph = workflow.compile(checkpointer = memory)

In [75]:
# visualize the graph

# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_mermaid_png()))

In [76]:
config = {"configurable": {"thread_id": "0"}}
from pprint import pp
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Hi! can you make the following text all upper case: 'banana' and tell me what 2*2 is?")
        ]
    }, config=config
):
    if "__end__" not in s:
        pp(s)
        print("----")

{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----
{'Supervisor': {'next': 'TextHandler'}}
----
{'TextHandler': None}
----


KeyboardInterrupt: 